In [ ]:
# "True": print info useful for debugging purposes,
# "False": less-verbose, better-suited for training.
print_debugging_info = False

# "True": monitor training progress,
# "False": maximum performance.
print_training_info = True

# "True": define a test set on which to evaluate the model,
# "False": use all available data to train.
evaluate_the_model = False

# 🌐 Mount Google drive

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 1

Mounted at /gdrive
/gdrive/My Drive/[2024-2025] AN2DL/Homework 1


# ⚙️ Import libraries

In [ ]:
# These are being used both at design time (here) and at test time (when prediction is executed on codabench)
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import callbacks as tfkc

# These are being used only at design time
import gc
import random
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

# These need to be installed
!pip install keras-cv
import keras_cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.0 MB/s eta 0:00:00


# ⏳ Load, inspect and clean up the dataset

In [ ]:
# Load the dataset
data = np.load('Balanced_dataset.npz')
X = data['arr_0']
y = data['arr_1']

# Shuffle the dataset
indices = np.random.permutation(X.shape[0])
X = X[indices]
y = y[indices]

# Define a mapping of labels to their corresponding digit names
labels = {0:'Basophil', 1:'Eosinophil', 2:'Erythroblast', 3:'Immature granulocytes', 4:'Lymphocyte', 5:'Monocyte', 6:'Neutrophil', 7:'Platelet'}

# Save unique labels
unique_labels = list(labels.values())

In [ ]:
if print_debugging_info == True:
  # Print the shapes of the loaded datasets
  print("Training-Validation Data Shape:", X.shape)
  print("Training-Validation Label Shape:", y.shape)

  # Inspect the target
  print('Counting occurrences of target classes:')
  print(pd.DataFrame(y, columns=['digit'])['digit'].value_counts())

  # Display a sample of images from the dataset
  num_img = 10
  random_indices = random.sample(range(len(X)), num_img)

  fig, axes = plt.subplots(1, num_img, figsize=(20, 20))

  # Iterate through the selected number of images
  for i, idx in enumerate(random_indices):
      ax = axes[i % num_img]
      ax.imshow(np.squeeze(X[idx]), vmin=0., vmax=1.)
      ax.set_title(f'{labels[y[idx][0]]}')
      ax.axis('off')

  # Adjust layout and display the images
  plt.tight_layout()
  plt.show()

In [ ]:
# Convert labels to categorical format using one-hot encoding
y = tfk.utils.to_categorical(y)

test_set_size = 0.1
fine_tuning_validation_set_size = 0.1

# Split data into training, validation and test sets, maintaining class distribution
if evaluate_the_model == True:
  X_train_ft, X_test, y_train_ft, y_test = train_test_split(X, y, test_size=test_set_size, stratify=y)
  X_train, X_ft, y_train, y_ft = train_test_split(X_train_ft, y_train_ft, test_size=fine_tuning_validation_set_size, stratify=y_train_ft)
else:
  X_train, X_ft, y_train, y_ft = train_test_split(X, y, test_size=fine_tuning_validation_set_size, stratify=y)

In [ ]:
if print_debugging_info == True:
  # Print the shapes of the resulting datasets
  print("Training Data Shape:", X_train.shape)
  print("Training Label Shape:", y_train.shape)
  print("Fine-tuning Data Shape:", X_ft.shape)
  print("Fine-tuning Label Shape:", y_ft.shape)
  if evaluate_the_model == True:
    print("Test Data Shape:", X_test.shape)
    print("Test Label Shape:", y_test.shape)

# 🔧 Define the augmentation pipeline to enhance the dataset

In [ ]:
# Define the augmentation functions
aug1 = keras_cv.layers.AugMix((0,255))
aug2 = keras_cv.layers.RandAugment((0,255))

X_ft = aug1(aug2(X_ft))

In [ ]:
# Define an augmentation pipeline (not part of the real model)
# to visualize the augmentations that are applied at test time
if print_debugging_info == True:
  augmentation = tf.keras.Sequential([
    aug1,
    aug2
  ], name='preprocessing')

  num_img = 10
  random_indices = random.sample(range(len(X)), num_img)

  image_width = 2
  image_height = 2
  cols = 10
  rows = 2

  fig_width = cols * image_width
  fig_height = rows * image_height
  figsize = (fig_width, fig_height)

  fig, axes = plt.subplots(2, num_img, figsize=figsize, tight_layout=True)
  chosen_images = np.empty((0, 96, 96, 3))

  # Iterate through the selected number of images
  for i, idx in enumerate(random_indices):
      ax = axes[0, i % num_img]
      ax.imshow(np.squeeze(X[idx]), aspect='auto')
      ax.set_title(f'{idx}\n{labels[np.argmax(y[idx])]}', y=1.0, fontsize=16, pad=-32)
      ax.axis('off')

      chosen_images = np.append(chosen_images, np.expand_dims(X[idx], axis=0), axis=0)

  chosen_images = augmentation(chosen_images, training=True) / 255

  for i, image in enumerate(chosen_images):
      ax = axes[1, i % num_img]
      ax.imshow(np.squeeze(image), aspect='auto')
      ax.set_title('Augmented', y=1.0, fontsize=16, pad=-16)
      ax.axis('off')

  # Adjust layout and display the images
  plt.show()

# 📉 Define a new loss function

In [ ]:
@tf.keras.utils.register_keras_serializable()
# Smooth the labels in order to avoid overfitting
def smoothed_labels(y_true, smoothing=0.05):
    num_classes = tf.cast(tf.shape(y_true)[-1], tf.float32)
    smooth_positives = 1.0 - smoothing
    smooth_negatives = smoothing / (num_classes - 1)
    return y_true * smooth_positives + smooth_negatives

@tf.keras.utils.register_keras_serializable()
# Let the model focus on challenging examples
def focal_loss(alpha=1, gamma=0.5):
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)  # Avoid log(0)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        return tf.reduce_mean(weight * cross_entropy)
    return loss_fn

@tf.keras.utils.register_keras_serializable()
# Combine Label Smoothing and Focal Loss
def combined_loss(smoothing=0.05, alpha=1, gamma=0.5):
    def loss_fn(y_true, y_pred):
        # Apply label smoothing
        y_true_smoothed = smoothed_labels(y_true, smoothing)
        # Apply focal loss
        return focal_loss(alpha, gamma)(y_true_smoothed, y_pred)
    return loss_fn

# 🧠 Build the *OverfittingModel* with transfer learning

In [ ]:
image_dim = (96, 96, 3)

# Initialise a feature extractor with pretrained weights for transfer learning
tl_fe = tfk.applications.ConvNeXtXLarge(
    input_shape = image_dim,

    pooling = None,
    include_top = False,
    include_preprocessing = True,

    weights = "imagenet",
    name = "convnext_xlarge",
)

# Freeze all layers in the feature extractor
tl_fe.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape = image_dim, name = 'input_layer')

# Pass inputs through the feature extractor
x = tl_fe(inputs)

classifier = tfkl.MaxPooling2D(name='max_pool')(x)

if print_debugging_info == True:
  # Define an inception block to use as a prediction head
  x1 = tfkl.Conv2D(128, kernel_size=1, padding='same', activation='relu')(classifier)

  x2 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
  x3 = tfkl.Conv2D(192, kernel_size=3, padding='same', activation='relu')(x2)

  x4 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
  x5 = tfkl.Conv2D(96, kernel_size=5, padding='same', activation='relu')(x4)

  x6 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(classifier)
  x7 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(x6)

  y = tfkl.Concatenate(axis=-1)([x1, x3, x5, x7])
  y = tfkl.GlobalAveragePooling2D(name='gap')(y)
  y = tfkl.Dropout(0.3)(y)
  outputs = tfkl.Dense(8, activation='softmax')(y)

  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
  tl_model.compile(loss=combined_loss(smoothing=0.05, alpha=1, gamma=0.5), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

  # Display a summary of the model architecture
  tl_model.summary(expand_nested=True)

  # Display model architecture with layer shapes and trainable parameters
  tfk.utils.plot_model(tl_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)
  del tl_model

1393257616/1393257616 ━━━━━━━━━━━━━━━━━━━━ 60s 0us/step


Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 96, 96, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_xlarge           │ (None, 3, 3, 2048)     │    348,147,968 │ input_layer[0][0]      │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └ input_layer_1        │ (None, 96, 96, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └                      │ (None, 96, 96, 3)      │              0 │ -                      │
│ convnext_xlarge_prestem_… │                        │                │                        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └ convnext_xlarge_stem │ (None, 24, 24, 256)    │         13,056 │ -                      │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│       └                   │ (None, 24, 24, 256)    │         12,544 │ -                      │
│ convnext_xlarge_stem_conv │                        │                │                        │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│       └                   │ (None, 24, 24, 256)    │            512 │ -                      │
│ convnext_xlarge_stem_lay… │                        │                │                        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └                      │ (None, 24, 24, 256)    │         12,800 │ -                      │
│ convnext_xlarge_stage_0_… │                        │                │                        │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └                      │ (None, 24, 24, 256)    │            512 │ -                      │
│ convnext_xlarge_stage_0_… │                        │                │                        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └                      │ (None, 24, 24, 1024)   │        263,168 │ -                      │
│ convnext_xlarge_stage_0_… │                        │                │                        │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│    └                      │ (None, 24, 24, 1024)   │              0 │ -                      │
│ convnext_xlarge_stage_0_… │                        │                │                        │
│ (Activation)              │                        │                │                        │
├──────────────────────

 Total params: 349,071,976 (1.30 GB)

 Trainable params: 924,008 (3.52 MB)

 Non-trainable params: 348,147,968 (1.30 GB)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.92756 to fit



# 📜 Find the ideal number of epochs to train for through K-fold cross validation

In [ ]:
# Define the number of folds for cross-validation
num_folds = 4

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True)

if print_debugging_info == True:
  print("Original dimensions of the training set")
  print(len(X_train))
  print(len(y_train))

  print("\nFold's dimensions")
  for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):
    print("New fold")
    print(len(X_train[train_idx]))
    print(len(y_train[train_idx]))
    print(len(X_train[valid_idx]))
    print(len(y_train[valid_idx]))

Original dimensions of the training set
16809
16809

Fold's dimensions
New fold
12606
12606
4203
4203
New fold
12607
12607
4202
4202
New fold
12607
12607
4202
4202
New fold
12607
12607
4202
4202


In [ ]:
crossval_patience = 10

# The early-stopping criterion can either be based on accuracy on the validation
# set (in which case we like the maximum), or on the mse (in which case we like the minimum).
crossval_es_monitor = 'val_accuracy'
# crossval_monitor = 'val_mse'
crossval_es_mode = 'max'
# crossval_es_mode = 'min'

# Initialize a list to store best epochs
max_epochs = 100
best_epochs = np.full(num_folds, max_epochs)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):

    if print_training_info == True:
      print("Starting training on fold #: {}".format(fold_idx+1))

    # Create and compile a new model to be trained on the current fold
    # note that the feature extractor will stay the same while the classifier is retrained
    x1 = tfkl.Conv2D(128, kernel_size=1, padding='same', activation='relu')(classifier)

    x2 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
    x3 = tfkl.Conv2D(192, kernel_size=3, padding='same', activation='relu')(x2)

    x4 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
    x5 = tfkl.Conv2D(96, kernel_size=5, padding='same', activation='relu')(x4)

    x6 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(classifier)
    x7 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(x6)

    y = tfkl.Concatenate(axis=-1)([x1, x3, x5, x7])
    y = tfkl.GlobalAveragePooling2D(name='gap')(y)
    y = tfkl.Dropout(0.3)(y)
    outputs = tfkl.Dense(8, activation='softmax')(y)

    tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
    tl_model.compile(loss=combined_loss(smoothing=0.05, alpha=1, gamma=0.5), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

    # Define early stopping callback
    early_stopping_callback = tfkc.EarlyStopping(
      monitor = crossval_es_monitor,
      mode = crossval_es_mode,
      patience = crossval_patience,
      restore_best_weights = True
    )

    # Define final accuracy callback
    final_accuracy_callback = tfkc.LambdaCallback(
        on_epoch_end=lambda epoch, logs: setattr(
            final_accuracy_callback,
            'final_accuracy',
            max(logs.get('accuracy'), getattr(final_accuracy_callback, 'final_accuracy', 0))
        )
    )

    # Augment the training set at test time
    dataset = tf.data.Dataset.from_tensor_slices((X_train[train_idx], y_train[train_idx])).map(lambda x,y: (aug1(aug2(x)), y), num_parallel_calls=tf.data.AUTOTUNE).batch(64, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

    # Train the model on the current fold
    tl_model.fit(
      dataset,
      validation_data = (X_train[valid_idx], y_train[valid_idx]),
      epochs = max_epochs,

      callbacks = [early_stopping_callback, final_accuracy_callback],
      verbose = 1 if print_training_info else 0
    )

    # Calculate the best epoch for early stopping
    stopped_epoch = early_stopping_callback.stopped_epoch

    if stopped_epoch >= crossval_patience:
      best_epochs[fold_idx] = stopped_epoch - crossval_patience + 1

    if print_training_info == True:
      kfold_accuracy = round(final_accuracy_callback.kfold_accuracy * 100, 2)
      print(f"kfold accuracy: {kfold_accuracy}%")

    # Free memory by deleting the model instance
    del tl_model

Starting training on fold #: 1
Epoch 1/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 126s 459ms/step - accuracy: 0.4308 - loss: 0.1956 - val_accuracy: 0.8927 - val_loss: 0.0644
Epoch 2/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 56s 276ms/step - accuracy: 0.6974 - loss: 0.1136 - val_accuracy: 0.8929 - val_loss: 0.0615
Epoch 3/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 56s 275ms/step - accuracy: 0.7184 - loss: 0.1063 - val_accuracy: 0.9158 - val_loss: 0.0591
Epoch 4/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 55s 273ms/step - accuracy: 0.7451 - loss: 0.1011 - val_accuracy: 0.9139 - val_loss: 0.0573
Epoch 5/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 56s 279ms/step - accuracy: 0.7491 - loss: 0.0993 - val_accuracy: 0.9277 - val_loss: 0.0543
Epoch 6/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 55s 273ms/step - accuracy: 0.7631 - loss: 0.0960 - val_accuracy: 0.9058 - val_loss: 0.0568
Epoch 7/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 55s 272ms/step - accuracy: 0.7565 - loss: 0.0969 - val_accuracy: 0.9272 - val_loss: 0.0531
Epoch 8/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 55s

In [ ]:
# Collect garbage in case some memory was leaked to reduce RAM usage
gc.collect()

62723

In [ ]:
# Calculate the average best epoch
avg_epochs = max(1, int(np.mean(best_epochs)))
print(f"Best average epoch: {avg_epochs}")

Best average epoch: 29


# 🏋🏻‍♀️ Final training

In [ ]:
# Repeat, for the last time, the steps required to build the model
x1 = tfkl.Conv2D(128, kernel_size=1, padding='same', activation='relu')(classifier)

x2 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
x3 = tfkl.Conv2D(192, kernel_size=3, padding='same', activation='relu')(x2)

x4 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(classifier)
x5 = tfkl.Conv2D(96, kernel_size=5, padding='same', activation='relu')(x4)

x6 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(classifier)
x7 = tfkl.Conv2D(64, kernel_size=1, padding='same', activation='relu')(x6)

y = tfkl.Concatenate(axis=-1)([x1, x3, x5, x7])
y = tfkl.GlobalAveragePooling2D(name='gap')(y)
y = tfkl.Dropout(0.3)(y)
outputs = tfkl.Dense(8, activation='softmax')(y)

tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
tl_model.compile(loss=combined_loss(smoothing=0.05, alpha=1, gamma=0.5), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Define final accuracy callback
final_accuracy_callback = tfkc.LambdaCallback(
    on_epoch_end=lambda epoch, logs: setattr(
        final_accuracy_callback,
        'final_accuracy',
        max(logs.get('accuracy'), getattr(final_accuracy_callback, 'final_accuracy', 0))
    )
)

# Augment the training set at test time
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(lambda x,y: (aug1(aug2(x)), y), num_parallel_calls=tf.data.AUTOTUNE).batch(64, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

# Train the final model on the combined training and validation data
tl_model.fit(
    dataset,
    epochs = avg_epochs,

    callbacks = [final_accuracy_callback],
    verbose = 1 if print_training_info else 0
)

# Calculate and print the best accuracy achieved
final_tl_accuracy = round(final_accuracy_callback.final_accuracy * 100, 2)
print(f'Final accuracy (on the whole dataset) of the non fine-tuned model: {final_tl_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
tl_model.save(f'OverfittingModel_{str(final_tl_accuracy)}.keras')

# Free memory by deleting the model instance
del tl_model

Epoch 1/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 106s 313ms/step - accuracy: 0.4788 - loss: 0.1807
Epoch 2/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 265ms/step - accuracy: 0.7067 - loss: 0.1101
Epoch 3/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 263ms/step - accuracy: 0.7325 - loss: 0.1039
Epoch 4/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 262ms/step - accuracy: 0.7556 - loss: 0.0992
Epoch 5/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 262ms/step - accuracy: 0.7585 - loss: 0.0967
Epoch 6/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 263ms/step - accuracy: 0.7640 - loss: 0.0959
Epoch 7/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 263ms/step - accuracy: 0.7729 - loss: 0.0930
Epoch 8/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 70s 261ms/step - accuracy: 0.7767 - loss: 0.0931
Epoch 9/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 262ms/step - accuracy: 0.7836 - loss: 0.0915
Epoch 10/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 70s 261ms/step - accuracy: 0.7767 - loss: 0.0913
Epoch 11/29
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 263ms/step - accuracy: 0.7805 - loss: 0.0909
Epoch 12/29
263/26

In [ ]:
# Collect garbage in case some memory was leaked to reduce RAM usage
gc.collect()

22968

# 👨🏻‍💻 Fine-tuning

In [ ]:
# Re-load the model
ft_model = tfk.models.load_model(f'OverfittingModel_{str(final_tl_accuracy)}.keras', custom_objects={'loss_fn': combined_loss})

if print_debugging_info == True:
  # Display model architecture with layer shapes and trainable parameters
  tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Display debugging info about trainable layers
def display_trainable_layers_info(model, verbose = False):
  trainable = 0
  nontrainable = 0
  for i, layer in enumerate(model.get_layer('convnext_xlarge').layers):
    if verbose == True:
      print(i, layer.name, type(layer).__name__, layer.trainable)

    if layer.trainable == True:
      trainable = trainable + 1
    else:
      nontrainable = nontrainable + 1

  print(f'Number of trainable layers inside the feature extractor: {trainable},\nnumber of non-trainable layers inside the feature extractor: {nontrainable}.\n')

In [ ]:
# Enable training only for num_layers_to_unfreeze Conv2D and DepthwiseConv2D layers
def unfreeze_layers_v1(model, num_layers_to_unfreeze):
    unfrozen = 0
    for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            layer.trainable = True
            unfrozen += 1
            if unfrozen == num_layers_to_unfreeze:
                break

# Enable training only for Conv2D and DepthwiseConv2D layers present in the first num_layers_to_unfreeze layers
def unfreeze_layers_v2(model, num_layers_to_unfreeze):
    for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers[-1 * num_layers_to_unfreeze:]):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            layer.trainable = True

# Enable training only for num_layers_to_unfreeze Dense, Conv2D and DepthwiseConv2D layers
def unfreeze_layers_v3(model, num_layers_to_unfreeze):
    unfrozen = 0
    for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D) or isinstance(layer, tf.keras.layers.Dense):
            layer.trainable = True
            unfrozen += 1
            if unfrozen == num_layers_to_unfreeze:
                break

In [ ]:
# Fine-tuning of the convolutional layers

# Set every layer as non-trainable
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers[:]):
    layer.trainable = False

num_stages = 10
epochs_per_stage = 25
layers_to_unfreeze_per_stage = 10
ft_patience = 8
save_after_each_stage = True

crossval_es_monitor = 'val_accuracy'
crossval_es_mode = 'max'

# Define a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=100000,
    decay_rate=0.96
)

# Define early stopping callback
early_stopping_callback = tfkc.EarlyStopping(
    monitor = crossval_es_monitor,
    mode = crossval_es_mode,
    patience = ft_patience,
    restore_best_weights = True
)

# Define final accuracy callback
final_accuracy_callback = tfkc.LambdaCallback(
    on_epoch_end=lambda epoch, logs: setattr(
        final_accuracy_callback,
        'final_accuracy',
        max(logs.get('accuracy'), getattr(final_accuracy_callback, 'final_accuracy', 0))
    )
)

# Augment the training set at test time
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(lambda x,y: (aug1(aug2(x)), y), num_parallel_calls=tf.data.AUTOTUNE).batch(64, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

# Train with gradual unfreezing to fine-tune the model
for stage in range(num_stages):
    print(f"Stage {stage + 1}: Unfreezing {layers_to_unfreeze_per_stage} layers")

    # Unfreeze more layers
    unfreeze_layers_v1(ft_model, (stage + 1) * layers_to_unfreeze_per_stage)

    # Compile the model with the learning rate schedule
    ft_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=combined_loss(smoothing=0.05, alpha=1, gamma=0.5),
        metrics=['accuracy']
    )

    if print_training_info == True:
        display_trainable_layers_info(ft_model)

    # Train
    ft_model.fit(
        dataset,
        epochs = epochs_per_stage,
        validation_data = (X_ft, y_ft),

        callbacks = [early_stopping_callback, final_accuracy_callback],
        verbose = 1 if print_training_info else 0
    )

    # Calculate and print the current validation accuracy
    current_ft_val_accuracy = round(final_accuracy_callback.final_accuracy * 100, 2)
    print(f'Current validation accuracy of the fine-tuned model: {current_ft_val_accuracy}%\n')

    if save_after_each_stage == True:
      # Save the trained model to a file with the accuracy included in the filename
      ft_model.save(f'OverfittingModel_{str(current_ft_val_accuracy)}.keras')

Stage 1: Unfreezing 10 layers
Number of trainable layers inside the feature extractor: 10,
number of non-trainable layers inside the feature extractor: 249.

Epoch 1/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 175s 474ms/step - accuracy: 0.8119 - loss: 0.0821 - val_accuracy: 0.8522 - val_loss: 0.0699
Epoch 2/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 83s 308ms/step - accuracy: 0.8381 - loss: 0.0750 - val_accuracy: 0.8608 - val_loss: 0.0672
Epoch 3/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 82s 305ms/step - accuracy: 0.8490 - loss: 0.0727 - val_accuracy: 0.8694 - val_loss: 0.0659
Epoch 4/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 82s 307ms/step - accuracy: 0.8516 - loss: 0.0715 - val_accuracy: 0.8737 - val_loss: 0.0648
Epoch 5/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 83s 308ms/step - accuracy: 0.8469 - loss: 0.0729 - val_accuracy: 0.8774 - val_loss: 0.0647
Epoch 6/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 82s 307ms/step - accuracy: 0.8614 - loss: 0.0695 - val_accuracy: 0.8758 - val_loss: 0.0641
Epoch 7/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 81s 303ms/step 

In [ ]:
# Fine-tuning of the dense layers

# Set every layer as non-trainable
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers[:]):
    layer.trainable = False

num_stages = 3
epochs_per_stage = 20
layers_to_unfreeze_per_stage = 75
ft_patience = 5

# Editing the weights of convolutional layers turns them rom float16 to float32,
# this makes the model weigh a lot more, so not saving it at each stage is better.
save_after_each_stage = False

crossval_es_monitor = 'val_accuracy'
crossval_es_mode = 'max'

# Define a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=100000,
    decay_rate=0.96
)

# Define early stopping callback
early_stopping_callback = tfkc.EarlyStopping(
    monitor = crossval_es_monitor,
    mode = crossval_es_mode,
    patience = ft_patience,
    restore_best_weights = True
)

# Define final accuracy callback
final_accuracy_callback = tfkc.LambdaCallback(
    on_epoch_end=lambda epoch, logs: setattr(
        final_accuracy_callback,
        'final_accuracy',
        max(logs.get('accuracy'), getattr(final_accuracy_callback, 'final_accuracy', 0))
    )
)

# Augment the training set at test time
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(lambda x,y: (aug1(aug2(x)), y), num_parallel_calls=tf.data.AUTOTUNE).batch(64, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

# Train with gradual unfreezing to fine-tune the model
for stage in range(num_stages):
    print(f"Stage {stage + 1}: Unfreezing {layers_to_unfreeze_per_stage} layers")

    # Unfreeze more layers
    unfreeze_layers_v3(ft_model, (stage + 1) * layers_to_unfreeze_per_stage)

    # Compile the model with the learning rate schedule
    ft_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=combined_loss(smoothing=0.05, alpha=1, gamma=0.5),
        metrics=['accuracy']
    )

    if print_training_info == True:
        display_trainable_layers_info(ft_model)

    # Train
    ft_model.fit(
        dataset,
        epochs = epochs_per_stage,
        validation_data = (X_ft, y_ft),

        callbacks = [early_stopping_callback, final_accuracy_callback],
        verbose = 1 if print_training_info else 0
    )

    # Calculate and print the current validation accuracy
    current_ft_val_accuracy = round(final_accuracy_callback.final_accuracy * 100, 2)
    print(f'Current validation accuracy of the fine-tuned model: {current_ft_val_accuracy}%\n')

    if save_after_each_stage == True:
      # Save the trained model to a file with the accuracy included in the filename
      ft_model.save(f'OverfittingModel_{str(current_ft_val_accuracy)}.keras')

Stage 1: Unfreezing 75 layers
Number of trainable layers inside the feature extractor: 75,
number of non-trainable layers inside the feature extractor: 184.

Epoch 1/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 208s 524ms/step - accuracy: 0.9354 - loss: 0.0486 - val_accuracy: 0.9524 - val_loss: 0.0436
Epoch 2/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 87s 326ms/step - accuracy: 0.9396 - loss: 0.0472 - val_accuracy: 0.9556 - val_loss: 0.0441
Epoch 3/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 86s 322ms/step - accuracy: 0.9451 - loss: 0.0465 - val_accuracy: 0.9561 - val_loss: 0.0433
Epoch 4/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 86s 321ms/step - accuracy: 0.9478 - loss: 0.0454 - val_accuracy: 0.9545 - val_loss: 0.0422
Epoch 5/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 87s 324ms/step - accuracy: 0.9470 - loss: 0.0456 - val_accuracy: 0.9588 - val_loss: 0.0414
Epoch 6/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 88s 328ms/step - accuracy: 0.9564 - loss: 0.0436 - val_accuracy: 0.9663 - val_loss: 0.0406
Epoch 7/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 86s 320ms/step 

In [ ]:
# Calculate and print the final validation accuracy
final_ft_val_accuracy = round(final_accuracy_callback.final_accuracy * 100, 2)
print(f'Final validation accuracy of the fine-tuned model: {final_ft_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
ft_model.save(f'OverfittingModel_{str(final_ft_val_accuracy)}.keras')

Final validation accuracy of the fine-tuned model: 96.95%


In [ ]:
# Collect garbage in case some memory was leaked to reduce RAM usage
gc.collect()

20384

# 📟 Evaluate the model

In [ ]:
# Define the test-time augmentation procedure
def test_time_augment(image):
    augmented_images = []
    augmentations = [
        tf.image.flip_left_right,
        tf.image.flip_up_down,
        lambda x: tf.image.rot90(x, k=1),
        lambda x: tf.image.rot90(x, k=2)
    ]

    for augment in augmentations:
        augmented_images.append(augment(image))

    return augmented_images

# Augment the test images in batches
def predict_with_tta_batch(model, test_images, tta_function, batch_size=32):
    final_predictions = []

    for image in test_images:
        augmented_images = tta_function(image)
        augmented_images = tf.stack(augmented_images, axis=0)
        predictions = model.predict(augmented_images, batch_size=batch_size, verbose=0)
        mean_prediction = np.mean(predictions, axis=0)
        final_predictions.append(mean_prediction)

    return np.array(final_predictions)

# Test the model and print evaluation metrics
if evaluate_the_model == True:
    predictions = predict_with_tta_batch(ft_model, X_test, test_time_augment)
    predictions = np.argmax(predictions, axis=-1)
    test_gt = np.argmax(y_test, axis=-1)

    test_accuracy = accuracy_score(test_gt, predictions)
    print(f'Test accuracy: {test_accuracy * 100:.2f}%')

    not_hot_encoded_labels = {'Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes', 'Lymphocyte', 'Monocyte', 'Neutrophil', 'Platelet'}
    print(classification_report(test_gt, predictions, target_names=not_hot_encoded_labels))

    cm = confusion_matrix(test_gt, predictions)
    print(cm)

In [ ]:
# Delete the model to free up resources
del ft_model

In [ ]:
# Collect garbage in case some memory was leaked to reduce RAM usage
gc.collect()

0

# 📲 Prepare submission


In [ ]:
%%writefile model.py
import numpy as np
import glob

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

def test_time_augment(image):
    augmented_images = []
    augmentations = [
        tf.image.flip_left_right,
        tf.image.flip_up_down,
        lambda x: tf.image.rot90(x, k=1),
        lambda x: tf.image.rot90(x, k=2)
    ]

    for augment in augmentations:
        augmented_images.append(augment(image))

    return augmented_images

def predict_with_tta_batch(model, test_images, tta_function, batch_size=32):
    final_predictions = []

    for image in test_images:
        augmented_images = tta_function(image)
        augmented_images = tf.stack(augmented_images, axis=0)
        predictions = model.predict(augmented_images, batch_size=batch_size, verbose=0)
        mean_prediction = np.mean(predictions, axis=0)
        final_predictions.append(mean_prediction)

    return np.array(final_predictions)

@tf.keras.utils.register_keras_serializable()
def smoothed_labels(y_true, smoothing=0.05):
    num_classes = tf.cast(tf.shape(y_true)[-1], tf.float32)
    smooth_positives = 1.0 - smoothing
    smooth_negatives = smoothing / (num_classes - 1)
    return y_true * smooth_positives + smooth_negatives

@tf.keras.utils.register_keras_serializable()
def focal_loss(alpha=1, gamma=0.5):
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        return tf.reduce_mean(weight * cross_entropy)
    return loss_fn

@tf.keras.utils.register_keras_serializable()
def combined_loss(smoothing=0.05, alpha=1, gamma=0.5):
    def loss_fn(y_true, y_pred):
        y_true_smoothed = smoothed_labels(y_true, smoothing)
        return focal_loss(alpha, gamma)(y_true_smoothed, y_pred)
    return loss_fn

class Model:
    def __init__(self):
        # Define the pattern for the model file
        pattern = 'OverfittingModel*.keras'

        # Search for files that match the pattern
        model_files = glob.glob(pattern)

        if model_files:
          # Load the first model file that matches the pattern
          self.neural_network = tfk.models.load_model(model_files[0], custom_objects={'loss_fn': combined_loss})
        else:
          print('No model file found.')

    def predict(self, X):
        preds = predict_with_tta_batch(self.neural_network, X, test_time_augment)
        preds = np.argmax(preds, axis=-1)
        return preds

Overwriting model.py


In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py 'OverfittingModel'_{str(final_ft_val_accuracy)}.keras

#from google.colab import files
#files.download(filename)

  adding: model.py (deflated 63%)
  adding: OverfittingModel_96.95.keras (deflated 7%)


# 🔪 Kill the colab session

In [ ]:
%reset -f

import google.colab.runtime
google.colab.runtime.unassign()